# Le modèle:


### Definition du volume dans le drive:
Cette partie nous permettera de stocker le modèle sur notre drive dans le dossier precisé dans le code suivant.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Installation des bibliothèques nécessaires python avec pip:
Cette partie doit obligatoirement être executé pour le bon fonctionnement du programme.

In [ ]:
!pip install --upgrade keras-nlp
!pip install --upgrade keras
!pip install datasets
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━

In [ ]:
from datasets import load_dataset
import os
import keras
import keras_nlp
import tensorflow as tf
import time

Definition de la variable d'environnement de keras pour utiliser tensorflow dans le backend.

In [ ]:
os.environ["KERAS_BACKEND"] = "tensorflow"

Mise en place de dataset:

In [ ]:
dataset = load_dataset('financial_phrasebank', 'sentences_75agree')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Importation du modèle gpt 2

In [ ]:
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length = 128
)

gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en",
    preprocessor=preprocessor
)

100%|██████████| 448/448 [00:00<00:00, 392kB/s]
100%|██████████| 446k/446k [00:00<00:00, 1.25MB/s]
100%|██████████| 0.99M/0.99M [00:00<00:00, 2.37MB/s]
100%|██████████| 484/484 [00:00<00:00, 105kB/s]
100%|██████████| 475M/475M [00:17<00:00, 27.8MB/s]


In [ ]:
gpt2_lm.summary()

Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                     │                                              50,257 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃     Param # ┃ Connected to                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │           0 │ -                              │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │           0 │ -                              │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 768)         │ 124,439,808 │ padding_mask[0][0],            │
│                               │                           │             │ token_ids[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────┼────────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │  38,597,376 │ gpt2_backbone[0][0]            │
│ (ReversibleEmbedding)         │                           │             │                                │
└───────────────────────────────┴───────────────────────────┴─────────────┴────────────────────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_ds = dataset['train']['sentence']

num_epochs = 30

In [ ]:
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=len(train_ds) * num_epochs,
    end_learning_rate=0.0
)

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=['accuracy']
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3,
    mode='min',
    restore_best_weights=True
)

In [ ]:
gpt2_lm.fit(
    train_ds,
    epochs=num_epochs,
    callbacks=[early_stopping]
)

Epoch 1/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 333s 2s/step - accuracy: 0.3240 - loss: 0.9506
Epoch 2/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 127s 977ms/step - accuracy: 0.4115 - loss: 0.7614
Epoch 3/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 142s 981ms/step - accuracy: 0.4455 - loss: 0.6963
Epoch 4/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 141s 977ms/step - accuracy: 0.4704 - loss: 0.6505
Epoch 5/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 142s 979ms/step - accuracy: 0.4894 - loss: 0.6123
Epoch 6/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 142s 980ms/step - accuracy: 0.5104 - loss: 0.5778
Epoch 7/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 142s 981ms/step - accuracy: 0.5306 - loss: 0.5450
Epoch 8/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 106s 975ms/step - accuracy: 0.5496 - loss: 0.5138
Epoch 9/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 142s 980ms/step - accuracy: 0.5662 - loss: 0.4848
Epoch 10/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 142s 977ms/step - accuracy: 0.5861 - loss: 0.4558
Epoch 11/30
108/108 ━━━━━━━━━━━━━━━━━━━━ 106s 976ms/step - accuracy: 0.6022 - loss: 0.4283
Epoch 12/30

In [ ]:
gpt2_lm.save('/content/drive/MyDrive/Colab Notebooks/AI_SAE/GPT2_model.keras')